In [12]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
from tqdm import tqdm
import warnings
from keras.layers import *
from keras import Model

In [2]:
warnings.filterwarnings('ignore')
plt.rcParams["figure.dpi"] = 120
image_dimension = (224, 224)
image_input_size = (224, 224, 1)
epochs=100
batch_size = 32
epoch_index = [i for i in range(1, epochs+1)]

#### Load data

In [7]:
images = np.load(os.path.join("Brain_MRI_data1", "images.npy"), allow_pickle=True)
masks = np.load(os.path.join("Brain_MRI_data1", "masks.npy"), allow_pickle=True)

#### Change all image to grey

In [8]:
image_index = 0
for image in tqdm(images):
    plt.imsave(os.path.join("Brain_MRI_Segment_image", "image_{}.jpg".format(image_index)), image, cmap='gray')
    image_index += 1

mask_index = 0
for mask in tqdm(masks):
    plt.imsave(os.path.join("Brain_MRI_Segment_mask", "mask_{}.jpg".format(mask_index)), mask, cmap='gray')
    mask_index += 1

100%|██████████| 3064/3064 [00:16<00:00, 184.35it/s]


In [14]:
image_dir_name = "Brain_MRI_Segment_image"
mask_dir_name = "Brain_MRI_Segment_mask"
image_list = list()
mask_list = list()

images_dir = sorted(os.listdir(image_dir_name))
for image in tqdm(images_dir):
    current_image = cv2.imread(os.path.join(image_dir_name, image), cv2.IMREAD_GRAYSCALE)
    current_image = cv2.resize(current_image, image_dimension)
    current_image_180 = cv2.rotate(current_image, cv2.ROTATE_180)
    current_image_r90 = cv2.rotate(current_image, cv2.ROTATE_90_CLOCKWISE)
    current_image_l90 = cv2.rotate(current_image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    image_list.append(current_image)
    image_list.append(current_image_180)
    image_list.append(current_image_r90)
    image_list.append(current_image_l90)



masks_dir = sorted(os.listdir(mask_dir_name))
for mask in tqdm(masks_dir):
    current_mask = cv2.imread(os.path.join(mask_dir_name, mask), cv2.IMREAD_GRAYSCALE)
    current_mask = cv2.resize(current_mask, image_dimension)
    current_mask_180 = cv2.rotate(current_mask, cv2.ROTATE_180)
    current_mask_r90 = cv2.rotate(current_mask, cv2.ROTATE_90_CLOCKWISE)
    current_mask_l90 = cv2.rotate(current_mask, cv2.ROTATE_90_COUNTERCLOCKWISE)
    mask_list.append(current_mask)
    mask_list.append(current_mask_180)
    mask_list.append(current_mask_r90)
    mask_list.append(current_mask_l90)


features = np.array(image_list)/255.0
masks = np.array(mask_list)/255.0
features = features.reshape((features.shape[0], features.shape[1], features.shape[2], 1))
masks = masks.reshape((masks.shape[0], masks.shape[1], masks.shape[2], 1))

100%|██████████| 3064/3064 [00:00<00:00, 3860.11it/s]


In [15]:
print("image dimension: {}, label dimension: {}".format(features.shape, masks.shape))

image dimension: (12256, 224, 224, 1), label dimension: (12256, 224, 224, 1)


#### Train, Test, Validation Split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(features, masks, test_size=0.25, random_state=42, shuffle=True)
train_generator = ImageDataGenerator(validation_split=0.25)
test_generator = ImageDataGenerator()

train_df = train_generator.flow(X_train, y_train, batch_size=batch_size, subset="training")
validation_df = train_generator.flow(X_train, y_train, batch_size=batch_size, subset="validation", seed=42)
test_df = train_generator.flow(X_test, y_test, batch_size=batch_size)

#### U-Net Model Definition

In [17]:
def conv_block(inputs, filters, kernel_size=3, dropout_rate=0.2):
    x = tf.keras.layers.Conv2D(filters, kernel_size, padding='same', activation='relu')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.Conv2D(filters, kernel_size, padding='same', activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    return x

def unet(input_shape, num_classes):
    inputs = tf.keras.layers.Input(shape=input_shape)

    # Encoder path
    conv1 = conv_block(inputs, 32)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = conv_block(pool1, 64)
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = conv_block(pool2, 128)
    pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = conv_block(pool3, 256)
    pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4)
    conv5 = conv_block(pool4, 512)

    # Decoder path
    upconv6 = tf.keras.layers.Conv2DTranspose(256, kernel_size=2, strides=(2, 2), padding='same')(conv5)
    merge6 = tf.keras.layers.concatenate([upconv6, conv4], axis=3)
    conv6 = conv_block(merge6, 256)
    upconv7 = tf.keras.layers.Conv2DTranspose(128, kernel_size=2, strides=(2, 2), padding='same')(conv6)
    merge7 = tf.keras.layers.concatenate([upconv7, conv3], axis=3)
    conv7 = conv_block(merge7, 128)
    upconv8 = tf.keras.layers.Conv2DTranspose(64, kernel_size=2, strides=(2, 2), padding='same')(conv7)
    merge8 = tf.keras.layers.concatenate([upconv8, conv2], axis=3)
    conv8 = conv_block(merge8, 64)
    upconv9 = tf.keras.layers.Conv2DTranspose(32, kernel_size=2, strides=(2, 2), padding='same')(conv8)
    merge9 = tf.keras.layers.concatenate([upconv9, conv1], axis=3)
    conv9 = conv_block(merge9, 32)

    # Output layer
    outputs = tf.keras.layers.Conv2D(num_classes, 1, activation='sigmoid')(conv9)

    model = tf.keras.models.Model(inputs, outputs)
    return model

In [18]:
Unet = unet(image_input_size, 1)
Unet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
Unet.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_20 (Conv2D)             (None, 224, 224, 32  320         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_18 (BatchN  (None, 224, 224, 32  128        ['conv2d_20[0][0]']              
 ormalization)                  )                                                           

In [ ]:
unet_history = Unet.fit(train_df,
                        epochs=epochs,
                        verbose=1,
                        validation_data=validation_df)

Epoch 1/100


2023-02-18 16:06:21.214793: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
